# PFRA 

### _Development Notebook_
--- 

### QA/QC
#### Global Errors Check

This notebook is intended to identify global issues in a model simulation that require follow-up responses from the modeler before the model can be used in *STARRII* PFRA studies.

Global issues currently implemented include:
 
 __1. Check Model inputs__
 
    A. Plan Information
    B. Plan Parameters
    C. Boundary Conditions 
     
__2. Check Results for each domain (2D Flow Area) in the model__
 
    A. Computational Summary & Depth Plots
    B. Domain Extents
    C. High Velocities at cell faces
 
 
Global issues *in development* include:
 - Mapping areas where Courant number exceeds indicated threshold.
 - Mapping areas where iteration counts exceed indicated threshold.

In [7]:
model_s3path = "s3://azavea/jwx_test/data/zip/DC_F02_NBR_E0029_out.zip"

In [8]:
import sys;sys.path.append('../')

In [12]:
import gdal
from hecrasio.core import *
from hecrasio.qaqc import *
import scrapbook as sb
global_errors=[]

#### Read model output data 

*from AWS*

In [13]:
try:
    model = ResultsZip(model_s3path, pfra=False, require_prj=False)
except:
    global_errors.append('Failed to read model: ResultsZip error')
    
try:
    hdfResults_path = pull_result_paths(model)
except:
    global_errors.append('Failed to identify paths: hdfResults path error')
    
try:
    rasPlan = HDFResultsFile(model, model_s3path, hdfResults_path)
except:
    global_errors.append('Failed to read RAS plan: rasPlan error')

### 1A. Plan Information

In [ ]:
try:
    sb.glue(rasPlan.Plan_Information.to_json(), "json")
except:
    global_errors.append('Failed to glue Plan Information')

rasPlan.Plan_Information

### 1B. Plan Parameters

In [ ]:
try:
    sb.glue(rasPlan.Plan_Parameters.to_json(), "json")
except:
    global_errors.append('Failed to glue Plan Parameters')

rasPlan.Plan_Parameters

### 1C. Boundary Conditions 

#### Input hydrographs

In [ ]:
try:
    results_data = show_results(rasPlan.domains, model, rasPlan)
except:
    global_errors.append('Failed to Show Results')

try:
    sb.glue(results_data.to_json(), "json")
except:
    global_errors.append('Failed to glue Results Data')

### 2A. Computational Summary

In [ ]:
try:
    non_nan = rasPlan.summary.replace(np.nan, "00:00:00")
    sb.glue(non_nan.to_json(), "json")
except:
    global_errors.append('Failed to glue Plan Summary')

rasPlan.summary

In [ ]:
sb.glue(global_errors)

# END